In [3]:
# movie_recommender_app.py

import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# ---------------------- Step 1: Load and Preprocess Dataset ----------------------
# Load the dataset (use the correct path to your CSV)
movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# Keep only needed columns
movies = movies[['original_title', 'overview', 'tagline', 'genres']].dropna(subset=['original_title', 'overview'])
movies=movies.head(5000)
# Fill NaNs in tagline and genres
movies['tagline'] = movies['tagline'].fillna('')
movies['genres'] = movies['genres'].fillna('[]')

# Convert genre JSON string to space-separated genre names
def parse_genres(g):
    try:
        genres_list = ast.literal_eval(g)
        return " ".join([d['name'] for d in genres_list])
    except:
        return ""

movies['genres'] = movies['genres'].apply(parse_genres)

# Combine overview, genres, and tagline into one string for each movie
movies['combined'] = movies['overview'] + " " + movies['genres'] + " " + movies['tagline']

# ---------------------- Step 2: Vectorize and Compute Similarity ----------------------
tfidf = TfidfVectorizer(stop_words='english')
vectors = tfidf.fit_transform(movies['combined'])
similarity = cosine_similarity(vectors)

# ---------------------- Step 3: Recommender Function ----------------------
def recommend(movie):
    movie = movie.strip().lower()
    if movie not in movies['original_title'].str.lower().values:
        return [f"❌ Movie '{movie}' not found. Please try another."]

    idx = movies[movies['original_title'].str.lower() == movie].index[0]
    distances = list(enumerate(similarity[idx]))
    movie_list = sorted(distances, reverse=True, key=lambda x: x[1])[1:6]
    return [movies.iloc[i[0]]['original_title'] for i in movie_list]

# ---------------------- Step 4: Gradio Interface ----------------------
interface = gr.Interface(
    fn=recommend,
    inputs=gr.Textbox(label="Enter movie name", placeholder="e.g., Jumanji"),
    outputs=gr.Textbox(label="Recommended Movies"),
    title="🎬 Movie Recommender",
    description="Enter a movie name and get similar movie suggestions based on plot, genre, and tagline.",
    theme="dark"
)

interface.launch()


D:\Anaconda\Lib\site-packages\gradio\blocks.py:1096: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-686fefa0-7a33894808f3860342a65f42;e0e26dec-8607-4707-8eb4-5d85c3da1ed7)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
